# 도매꾹 의류 상품 크롤링 시스템

In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.webdriver.common.keys import Keys
import time
import random
import pandas as pd
import ast

### 판매가 공식

$${ 판매가 = {{수익 + 공급가} \over {1-수수료(퍼센트) - 리워드(퍼센트)}}}$$

$${ 십의 자리 반내림 }$$

In [2]:
PG_TAX_PERCENT = 0.018
REWARD_RATE = 0.03
PROFIT_RATE = 0.1
MARGIN_RATE = 0.1
CREATED_AT = "2024-03-29 00:00:00.000000"
UPDATED_AT = "2024-03-29 00:00:00.000000"

In [3]:
def round_ten(price):
    remainder = price % 10
    return price + (10 - remainder)

In [4]:
def get_sale_price(supply_price, reward_rate):
    sale_price = round((supply_price + supply_price * PROFIT_RATE) / (1 - PG_TAX_PERCENT - reward_rate))
    return round_ten(sale_price)

In [5]:
get_sale_price(19500,0.03)

22540

In [6]:
tax_status = {
    'TAX':'TAX',
	'ZERO_TAX_RATE':'ZERO_TAX_RATE',
	'TAX_EXEMPTION':'TAX_EXEMPTION'
}

In [7]:
product_status = {
    'ACTIVE':'ACTIVE',
	'OUT_OF_STOCK':'OUT_OF_STOCK',
	'SOLD_OUT':'SOLD_OUT',
	'DISCONTINUED':'DISCONTINUED'
}

In [3]:
brand_name = [
    {'name': 'Styloo', 'ko_name': '스타이루', 'id': 1},
    {'name': 'Serenity', 'ko_name': '세레니티', 'id': 2},
    {'name': 'Canvas Style', 'ko_name': '캔버스 스타일', 'id': 3},
    {'name': 'Lumina', 'ko_name': '루미나', 'id': 4},
    {'name': 'WOVEN', 'ko_name': '워벤', 'id': 5},
    {'name': 'Aether', 'ko_name': '에테르', 'id': 6},
    {'name': 'Bloomer', 'ko_name': '블루머', 'id': 7},
    {'name': 'Iris Eye', 'ko_name': '아이리스 아이', 'id': 8},
    {'name': 'Sweet Ander', 'ko_name': '스위트 앤더', 'id': 9},
    {'name': 'Prismer', 'ko_name': '프리즈머', 'id': 10},
    {'name': 'Sage Style', 'ko_name': '세이지 스타일', 'id': 11},
    {'name': 'Violet Bloom', 'ko_name': '바이올렛 블룸', 'id': 12},
    {'name': 'Clover Kiss', 'ko_name': '클로버 키스', 'id': 13},
    {'name': 'Honey Candy', 'ko_name': '허니 캔디', 'id': 14},
    {'name': 'Pluto', 'ko_name': '플루토', 'id': 15},
    {'name': 'CALLUNA', 'ko_name': '칼루나', 'id': 16},
    {'name': 'Twila', 'ko_name': '트와일라', 'id': 17},
    {'name': 'Velvetine', 'ko_name': '벨베틴', 'id': 18},
    {'name': 'AvaLys', 'ko_name': '아바리스', 'id': 19},
    {'name': 'VelaAva', 'ko_name': '벨라아바', 'id': 20}
]

In [9]:
option_key = {
    'COLOR':'COLOR',
	'SIZE':'SIZE',
	'TYPE':'TYPE'
}

In [10]:
def find_brand_by_name(brand_list):
    while True:
        input_name = input("브랜드 명을 작성해주세요. (RANDOM_BRAND: 무작위로 브랜드명 입력)");
        if "RANDOM_BRAND" == input_name:
            return brand_name[random.randint(0, len(brand_list) - 1)]
        for brand in brand_list:
            if brand['name'] == input_name:
                return brand
        print("해당 브랜드는 없습니다. 다시 작성해주세요.")

### 가져올 데이터
- productCategory: 제품 카테고리 ex) 아우터: (코트, 가디건,...), 상의(티셔츠, 맨투맨,...), 하의(데님 팬츠,...)
- 상품 브랜드 ex) 직접 입력
- 상품 옵션 ex) 빨강, 스웨터 레드
- 상품 이름 ex
- 상품 코드 ex
- 배송 정보 ex 당일출고 (평균출고일 0.4 일) 빠른배송
- 브랜드 이름
- 제조사 이름
- 공급사 이름
- 공급사 사이트 url
- 공급사 공급 가격
- 세일 비율
- 소비자 가격
- 판매 가격
- 세금 상태
- 소비자 리워드 비율
- 비즈니스 리워드 비율
- 상품 썸네일 이미지 경로
- 상품 상세 이미지 경로
- 상품 상태
- 상품 우선 순위: 직접 입력, 임의 지정
- 노출 유무
- 옵션 멀티 가능 유무
- 다 판매 시 타이틀
- 오늘 관출 조절 수

### 상품 옵션
- product_id
- sku_code
- option_key
- option_value
- option_sequence
- product_qty
- max_order_qty
- min_order_qty
- option_extra_amount
- is_exposed

In [11]:
product_option_data = {
    "product_id":"",
    "sku_code":"",
    "option_key":"",
    "option_value":"",
    "option_sequence":"",
    "product_qty":"",
    "max_order_qty":"",
    "min_order_qty":"",
    "option_extra_amount":"",
    "is_exposed":"",
}

### 예시
- http://domeggook.com/41493074?from=lstGen
- 사진 사용 여부: true, false
- 제품 카테고리 ex) 맨투맨 -> 번호
- 상품 브랜드 ex) 직접 입력, 임의 선택 : ["styloo", "feel good",...]
- 상품 옵션 ex) 블랙 XL, 그레이 115-XL : 블랙 
- 상품 이름 ex) 여자 오버핏 얇은 맨투맨 무지티 커플맨투맨 긴팔티 빅사이즈
- 상품 코드 ex) 형식에 맞게 자동 작성
- 배송 정보 ex) 당일출고 (평균출고일 0.4 일) 빠른배송
- 브랜드 이름 ex) 직접 입력
- 제조사 이름 ex) edinscompany2
- 공급사 이름 ex) 도매꾹
- 공급사 사이트 url ex) http://domeggook.com/41493074?from=lstGen
- 공급사 공급 가격 ex) 7900
- 세일 비율 e
- 소비자 가격
- 판매 가격
- 세금 상태 (TAX, ZERO_TAX_RATE, TAX_EXEMPTION)
- 소비자 리워드 비율
- 비즈니스 리워드 비율
- 상품 썸네일 이미지 경로
http://cdn1.domeggook.com/upload/item/2023/10/09/16968620268A659DB5C27F2BC4F60AF5/16968620268A659DB5C27F2BC4F60AF5_img_760?hash=fae76db097253dd6c028fc1a1bfb5c31
- 상품 상세 이미지 경로
https://superwhale.co.kr/web/upload/NNEditor/20231008/EC9687EC9D80EAB3B5EC9AA9EC98A4EBB284ED958FEBA7A8ED88ACEBA7A8-EC97ACEC84B1EB8F84EBA7A4.jpg
- 상품 상태: ex) (ACTIVE, OUT_OF_STOCK, SOLD_OUT, DISCONTINUED)
- 상품 우선 순위: 직접 입력, 임의 지정
- 노출 유무: ex) True, False
- 옵션 멀티 가능 유무: True, False
- 다 판매 시 타이틀: ex) 다 팔렸습니다.
- 오늘 관출 조절 수: ex) 0

In [13]:
data_frame = {
    'id': "",
    'isUsePhoto':"",
    'product_category_id':"",
    'product_brand_id':"",
    'productOptions':"",
    'name':"",
    'product_code':"",
    'shipping_info_id': "",
    'brand_name': "",
    'manufacturer_name': "",
    'supplier_name':"",
    'supplier_url': "",
    'supply_price': "",
    'discount_rate': "",
    'consumer_price': "",
    'sale_price': "",
    'tax_status': "",
    'consumer_reward_rate':"",
    'business_reward_rate':"",
    'image_path':"",
    'detail_image_path':"",
    'product_status':"",
    'product_priority':"",
    'is_exposed':"",
    'is_available_multiple_option':"",
    'sale_alternatives':"",
    'prod_obs_ctrl_num':"",
    'created_at': '',
    'updated_at': '',
    'tax_rate': '',
    'margin_rate': '',
}

In [14]:
data = {
    'id': [],
    'isUsePhoto':[],
    'product_category_id':[],
    'product_brand_id':[],
    'productOptions':[], # 리스트 형식
    'name':[],
    'product_code':[],
    'shipping_info_id': [],
    'brand_name': [],
    'manufacturer_name': [],
    'supplier_name':[],
    'supplier_url': [],
    'supply_price': [],
    'discount_rate': [],
    'consumer_price': [],
    'sale_price': [],
    'tax_status': [],
    'consumer_reward_rate':[],
    'business_reward_rate':[],
    'image_path':[],
    'detail_image_path':[],
    'product_status':[],
    'product_priority':[],
    'is_exposed':[],
    'is_available_multiple_option':[],
    'sale_alternatives':[],
    'prod_obs_ctrl_num':[],
    'created_at': [],
    'updated_at': [],
    'tax_rate': [],
    'margin_rate': [],
}

In [15]:
cloth_data_path = "./data/cloth_data/first_data/domeggok_cloth_data.csv"

try:
    df = pd.read_csv(cloth_data_path)
    data_record_list = df.to_dict(orient='records')
    for data_record in data_record_list:
        for key, value in data.items():
            if key == "productOptions":
                data[key].append(ast.literal_eval(data_record[key]))
            else:
                data[key].append(data_record[key])
except Exception as e:
    print(e)
    pass

[Errno 2] No such file or directory: './data/cloth_data/first_data/domeggok_cloth_data.csv'


In [16]:
data

{'id': [],
 'isUsePhoto': [],
 'product_category_id': [],
 'product_brand_id': [],
 'productOptions': [],
 'name': [],
 'product_code': [],
 'shipping_info_id': [],
 'brand_name': [],
 'manufacturer_name': [],
 'supplier_name': [],
 'supplier_url': [],
 'supply_price': [],
 'discount_rate': [],
 'consumer_price': [],
 'sale_price': [],
 'tax_status': [],
 'consumer_reward_rate': [],
 'business_reward_rate': [],
 'image_path': [],
 'detail_image_path': [],
 'product_status': [],
 'product_priority': [],
 'is_exposed': [],
 'is_available_multiple_option': [],
 'sale_alternatives': [],
 'prod_obs_ctrl_num': [],
 'created_at': [],
 'updated_at': [],
 'tax_rate': [],
 'margin_rate': []}

### 크롬 드라이버 연결

In [17]:
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("--mute-audio")
options.add_argument("incognito")
options.add_experimental_option("excludeSwitches",["enable-automation"])
options.add_experimental_option("excludeSwitches",["enable-logging"])

In [34]:
driver = webdriver.Chrome(options=options)

In [19]:
domeggook_url = "https://mirror1.domeggook.com/main/item/itemList.php"

In [20]:
domeggok_search_query_param = "?mnp=&mxp=&mnq=&mxq=&dfos=false&sgd=1&sw=&so=da"
domeggok_search_url = domeggook_url + domeggok_search_query_param
domeggokr_search_category = "&ca="

In [21]:
look_style_list = [
    {"id":0,"name":"캐주얼룩"},
    {"id":1,"name":"러블리룩"},
    {"id":2,"name":"스트릿룩"},
    {"id":3,"name":"미니멀룩"},
    {"id":4,"name":"오피스룩"},
    {"id":5,"name":"스포티룩"},
    {"id":6,"name":"사파리룩"},
    {"id":7,"name":"데이트룩"},
    {"id":8,"name":"레이어드룩"},
]

In [22]:
cloth_category = {
    "HOODIE":{"type":"HOODIE", "name":"후드티", "id":11},
	"SWEATER":{"type":"SWEATER", "name":"스웨터,니트", "id":12},
	"SHIRT":{"type":"SHIRT", "name":"셔츠", "id":13},
	"SWEATSHIRT":{"type":"SWEATSHIRT", "name":"맨투맨", "id":14,"look_style":[look_style_list[0],look_style_list[2]]},
	"T_SHIRT":{"type":"T_SHIRT", "name":"티셔츠", "id":15},
	"BLOUSE":{"type":"BLOUSE", "name":"블라우스", "id":16},
	"TANK_TOP":{"type":"TANK_TOP", "name":"민소매", "id":17},

    "CARDIGAN":{"type":"CARDIGAN", "name":"가디건", "id":21},
	"ZIP_UP_AND_JUMPER":{"type":"ZIP_UP_AND_JUMPER", "name":"집업,점퍼", "id":22},
	"SAFARI_JACKET":{"type":"SAFARI_JACKET", "name":"야상,사파리", "id":23},
	"JACKET":{"type":"JACKET", "name":"자켓", "id":24},
	"COAT":{"type":"COAT", "name":"코트", "id":25},

    "LONG_PANTS":{"type":"LONG_PANTS", "name":"롱팬츠", "id":31},
	"DENIM":{"type":"DENIM",  "name":"데님, 청바지", "id":32},
	"SLACKS":{"type":"SLACKS", "name":"슬랙스", "id":33},
	"STRAIGHT_PANTS":{"type":"STRAIGHT_PANTS", "name":"일자팬츠", "id":34},
	"SHORTS_PANTS":{"type":"SHORTS_PANTS",  "name":"숏팬츠, 반바지", "id":35},
	"SKINNY_JEANS":{"type":"SKINNY_JEANS", "name":"스키니", "id":36},

    "NIGHT_DRESS":{"type":"NIGHT_DRESS", "name":"롱 원피스", "id":41},
	"SHORT_DRESS":{"type":"SHORT_DRESS", "name":"미니 원피스", "id":42},
	"MIDI_DRESS":{"type":"MIDI_DRESS", "name":"미디 원피스", "id":43},
    
	"MINI_SKIRT":{"type":"MINI_SKIRT", "name":"미니 스커트", "id":51},
	"LONG_SKIRT":{"type":"LONG_SKIRT", "name":"롱 스커트", "id":52},
	"MIDI_SKIRT":{"type":"MIDI_SKIRT", "name":"미디 스커트", "id":53},

    "OTHERS":{"type":"OTHERS", "name":"직접 확인", "id":99}
}

In [23]:
seleceted_domeggok_search_params = [
    {"domeggok_title":"맨투맨티셔츠","category_param":'02_01_03_02_00', "cloth_category_id": [cloth_category["SWEATSHIRT"]],"look_style":[look_style_list[0],look_style_list[2]]},
    {"domeggok_title":"라운드넥티셔츠","category_param":'02_01_03_01_00', "cloth_category_id": [cloth_category["T_SHIRT"]],"look_style":[look_style_list[0],look_style_list[2],look_style_list[5]]},
    {"domeggok_title":"후드티셔츠","category_param":'02_01_03_08_00', "cloth_category_id": [cloth_category["HOODIE"]],"look_style":[look_style_list[0],look_style_list[1],look_style_list[2],look_style_list[3],look_style_list[5]]},
    {"domeggok_title":"탱크탑/나시티셔츠","category_param":'02_01_03_11_00', "cloth_category_id": [cloth_category["TANK_TOP"]],"look_style":[look_style_list[0],look_style_list[5]]},
    {"domeggok_title":"브이넥티셔츠","category_param":'02_01_03_03_00', "cloth_category_id": [cloth_category["T_SHIRT"]],"look_style":[look_style_list[0],look_style_list[2],look_style_list[5]]},
    {"domeggok_title":"터틀넥티셔츠","category_param":'02_01_03_06_00', "cloth_category_id": [cloth_category["T_SHIRT"]],"look_style":[]},
    {"domeggok_title":"소매롤업/7부티셔츠","category_param":'02_01_03_10_00', "cloth_category_id": [cloth_category["T_SHIRT"]],"look_style":[]},
    {"domeggok_title":"오프숄더티셔츠","category_param":'02_01_03_05_00', "cloth_category_id": [cloth_category["T_SHIRT"]],"look_style":[look_style_list[0],look_style_list[5],look_style_list[7]]},
    {"domeggok_title":"스퀘어넥티셔츠","category_param":'02_01_03_04_00', "cloth_category_id": [cloth_category["T_SHIRT"]],"look_style":[look_style_list[0],look_style_list[1],look_style_list[7]]},

    {"domeggok_title":"카라셔츠/블라우스","category_param":'02_01_04_02_00', "cloth_category_id": [cloth_category["BLOUSE"], cloth_category['SHIRT']],"look_style":[look_style_list[3],look_style_list[4]]},
    {"domeggok_title":"쉬폰/레이스블라우스","category_param":'02_01_04_04_00', "cloth_category_id": [cloth_category["BLOUSE"], cloth_category['SHIRT']],"look_style":[look_style_list[3],look_style_list[4]]},
    {"domeggok_title":"롱/원피스형블라우스","category_param":'02_01_04_01_00', "cloth_category_id": [cloth_category["BLOUSE"]],"look_style":[look_style_list[3],look_style_list[4]]},
    {"domeggok_title":"리본/셔링/프릴장식","category_param":'02_01_04_01_00', "cloth_category_id": [cloth_category["BLOUSE"]],"look_style":[look_style_list[3],look_style_list[4]]},
    {"domeggok_title":"청남방/데님셔츠","category_param":'02_01_04_06_00', "cloth_category_id": [cloth_category["BLOUSE"]],"look_style":[look_style_list[3],look_style_list[4]]},

    {"domeggok_title":"니트/스웨터","category_param":'02_01_05_00_00', "cloth_category_id": [cloth_category["SWEATER"]],"look_style":[look_style_list[0],look_style_list[1],look_style_list[3],look_style_list[8]]},

    {"domeggok_title":"기본자켓","category_param":'02_01_02_01_00', "cloth_category_id": [cloth_category["JACKET"]],"look_style":[look_style_list[3],look_style_list[8]]},
    {"domeggok_title":"베이직코트","category_param":'02_01_02_12_00', "cloth_category_id": [cloth_category["COAT"]],"look_style":[look_style_list[3],look_style_list[4],look_style_list[8]]},
    {"domeggok_title":"미니/볼레로","category_param":'02_01_02_07_00', "cloth_category_id": [cloth_category["CARDIGAN"]],"look_style":[look_style_list[0],look_style_list[1],look_style_list[7]]},
    {"domeggok_title":"트렌치코트","category_param":'02_01_02_11_00', "cloth_category_id": [cloth_category["COAT"]],"look_style":[look_style_list[3],look_style_list[4],look_style_list[8]]},
    {"domeggok_title":"사파리/야상점퍼","category_param":'02_01_02_10_00', "cloth_category_id": [cloth_category["SAFARI_JACKET"]],"look_style":[look_style_list[2],look_style_list[6],look_style_list[8]]},
    {"domeggok_title":"집업/점퍼/바람막이","category_param":'02_01_02_09_00', "cloth_category_id": [cloth_category["ZIP_UP_AND_JUMPER"]],"look_style":[look_style_list[0],look_style_list[2]]},
    {"domeggok_title":"니트자켓/코트","category_param":'02_01_02_06_00', "cloth_category_id": [cloth_category["OTHERS"]],"look_style":[]},
    {"domeggok_title":"후드자켓/점퍼","category_param":'02_01_02_02_00', "cloth_category_id": [cloth_category["ZIP_UP_AND_JUMPER"]],"look_style":[look_style_list[0],look_style_list[2],look_style_list[8],]},
    {"domeggok_title":"청자켓","category_param":'02_01_02_05_00', "cloth_category_id": [cloth_category["JACKET"]],"look_style":[look_style_list[0],look_style_list[2],]},
    {"domeggok_title":"가죽/라이더자켓","category_param":'02_01_02_04_00', "cloth_category_id": [cloth_category["JACKET"]],"look_style":[look_style_list[2]]},

    {"domeggok_title":"가디건/조끼","category_param":'02_01_06_00_00', "cloth_category_id": [cloth_category["CARDIGAN"]],"look_style":[look_style_list[0],look_style_list[1],look_style_list[3],look_style_list[8]]},
    
    {"domeggok_title":"원피스","category_param":'02_02_01_00_00', "cloth_category_id": [cloth_category["NIGHT_DRESS"], cloth_category['MIDI_DRESS'],cloth_category['MINI_SKIRT'],],"look_style":[look_style_list[1],look_style_list[7]]},
    {"domeggok_title":"미니원피스","category_param":'02_02_01_09_00', "cloth_category_id": [cloth_category['MINI_SKIRT'],],"look_style":[look_style_list[1],look_style_list[7]]},

    {"domeggok_title":"청바지","category_param":'02_02_03_00_00', "cloth_category_id": [cloth_category['DENIM'],],"look_style":[look_style_list[0],look_style_list[1],look_style_list[2],look_style_list[7],look_style_list[8]]},
    {"domeggok_title":"와이드/힙합청바지","category_param":'02_02_03_07_00', "cloth_category_id": [cloth_category['LONG_PANTS'],],"look_style":[look_style_list[0],look_style_list[2]]},
    {"domeggok_title":"반바지","category_param":'02_02_03_15_00', "cloth_category_id": [cloth_category['SHORTS_PANTS'],],"look_style":[look_style_list[0],look_style_list[2],look_style_list[5]]},
    {"domeggok_title":"바지&팬츠","category_param":'02_02_02_00_00', "cloth_category_id": [cloth_category['LONG_PANTS'],cloth_category['SHORTS_PANTS'],cloth_category['STRAIGHT_PANTS'],cloth_category['SLACKS'],cloth_category['DENIM'],cloth_category['SKINNY_JEANS']],"look_style":[look_style_list[0],look_style_list[2],look_style_list[5]]},
    {"domeggok_title":"일자바지","category_param":'02_02_02_01_00', "cloth_category_id": [cloth_category['STRAIGHT_PANTS']],"look_style":[look_style_list[0],look_style_list[3],look_style_list[4]]},
    {"domeggok_title":"통/와이드바지","category_param":'02_02_02_04_00', "cloth_category_id": [cloth_category['LONG_PANTS']],"look_style":[]},
    {"domeggok_title":"허리밴딩팬츠","category_param":'02_02_02_09_00', "cloth_category_id": [cloth_category['LONG_PANTS']],"look_style":[look_style_list[0]]},
    {"domeggok_title":"핫팬츠/반바지","category_param":'02_02_02_14_00', "cloth_category_id": [cloth_category['SHORTS_PANTS']],"look_style":[look_style_list[0],look_style_list[2],look_style_list[5]]},
    {"domeggok_title":"스키니진","category_param":'02_02_02_02_00', "cloth_category_id": [cloth_category['SKINNY_JEANS']],"look_style":[look_style_list[0],look_style_list[3],look_style_list[4]]},
    {"domeggok_title":"정장바지","category_param":'02_02_02_13_00', "cloth_category_id": [cloth_category['SLACKS']],"look_style":[look_style_list[0],look_style_list[3],look_style_list[4]]},
    {"domeggok_title":"카고바지","category_param":'02_02_02_06_00', "cloth_category_id": [cloth_category['LONG_PANTS']],"look_style":[look_style_list[2]]},

    {"domeggok_title":"치마/스커트","category_param":'02_02_04_00_00', "cloth_category_id": [cloth_category['LONG_SKIRT'],cloth_category['MINI_SKIRT'],cloth_category['MIDI_SKIRT']],"look_style":[look_style_list[2]]},
]

In [24]:
business_individually_purchas_name = ["사업자 낱개구매가능", "사업자낱개구매가능"]
individually_purchas_name = "낱개구매가능"
use_photo_title = "사용허용"
consumer_reward_rate_range = [0.0, REWARD_RATE]

sale_rate_range = [0.0,1.0]

In [25]:
def select_cloth_category(clothCategoryList):
    while True:
        for clothCategory in clothCategoryList['cloth_category_id']:
            print('id:',clothCategory['id'],'종류:',clothCategory['type'])

        isOtherCategory = False
        while True:
            try:
                isOtherCategory = int(input("이외 다른 의류 카테고리도 보시겠습니까? 0: No, 1: Yes"))
                if (isOtherCategory == 0):
                    break
                elif (isOtherCategory == 1):
                    isOtherCategory = True
                    break
                else:
                    print("잘못 입력 했습니다.")
            except Exception as e:
                print("잘못 입력 했습니다.")
                pass

        if (isOtherCategory):
            for key, value in cloth_category.items():
                print('id:',value['id'],'종류:',value['type'])
            input_id = int(input('id 번호 입력:  (-1 입력시 다시 처음으로 돌아가기)'))

            if (input_id == -1):
                continue
                
            for key, value in cloth_category.items():
                if value['id'] == input_id:
                    return value
            print("id 번호가 잘못 됐습니다.\n")
            
        
        else:
            try:
                input_id = int(input('id 번호 입력:  (-1 입력시 다시 처음으로 돌아가기)'))
                if (input_id == -1):
                    continue
            
                for item in clothCategoryList['cloth_category_id']:
                    if item['id'] == input_id:
                        return item
                    print("id 번호가 잘못 됐습니다.\n")
            except Exception as e:
                print("잘못 입력 했습니다.")
                pass

In [26]:
def convert_string_to_int_with_commas(string_number):
    # 문자열에서 쉼표 제거 후 정수로 변환
    number = int(string_number.replace(',', ''))
    return number

In [27]:
def create_product_code(category_type_id, category_data, data_num):
    '''
    인수 데이터 형식
    {'type': 'SWEATSHIRT',
     'name': '맨투맨',
     'id': 14,
     'look_style': [{'id': 0, 'name': '캐주얼룩'}, {'id': 2, 'name': '스트릿룩'}]}
    '''
    return int(str(category_type_id)+'{:03d}'.format(category_data['id'])+'{:08d}'.format(data_num + 1))

In [28]:
def check_range(value,rate_range):
    while True:
        if rate_range[0] <= value <= rate_range[1]:
            return value
        else:
            print("잘못 된 비율 입니다. ",value)

In [29]:
def get_priority(data):
    try:
        input_priority = int(input('우선 순위 번호를 입력해주세요. (-1: 무작위 수, 그외 숫자: 직접입력'))
        if (input_priority == -1):
            priority_min = (len(str(len(data['product_priority'])))) * 10
            priority_max = (len(str(len(data['product_priority'])))) * 100
            num = random.randint(priority_min,priority_max)
            return num
        
        if input_priority in data['product_priority']:
            return input_priority
    except Exception as e:
        print("잘못 입력 했습니다.")
        pass

In [30]:
def get_item_name(name):
    print("상품 이름:",name)
    while True:
        try:
            is_selected = int(input("해당 상품 이름으로 하시곘습니까? 1: Yes, 0: No"))
            if is_selected == 1:
                return name
            elif is_selected == 0:
                name = input("상품 이름을 넣어 주세요.")
                name_confirm = int(input("해당 상품 이름으로 확정하시겠습니까? 1: Yes, 0: No"))
                if name_confirm == 1:
                    return name
            else:
                print("잘못된 입력입니다. 다시 입력해주세요.")
        except Exception as e:
            pass

In [35]:
def main_func():
    for s_cloth_category in seleceted_domeggok_search_params:
        res = driver.get(f'{domeggok_search_url}{domeggokr_search_category}{s_cloth_category['category_param']}')
        category_page_num = int(driver.find_element(By.CSS_SELECTOR,"#lPage > form > span > b").text)
    
        for num in range(category_page_num):
            path = f'{domeggok_search_url}{domeggokr_search_category}{s_cloth_category['category_param']}&pg={num+1}'
            print(path)
            res = driver.get(path)
            elements = driver.find_elements(By.CSS_SELECTOR,"#lLst .lItemList")
            is_seeing = int(input("해당 페이지를 보시겠습니까? Yes:1, No:0, Next Cloth: -1"))
            if is_seeing == 0:
                continue
            if is_seeing == -1:
                break
                            
            for element in elements:
                
                items = element.find_elements(By.CSS_SELECTOR,"li")
        
                for item in items:
                    try:
                        # child_elements = item.find_elements(By.CSS_SELECTOR,"*")
                        # print("Child elements of li element:", [element for element in child_elements])
                        #li42340503 > div.amtqty > div.amt > b
                        title_element = item.find_element(By.CSS_SELECTOR,"div.main")
                        href_element = item.find_element(By.CSS_SELECTOR,"div.main a")
                        amt_element = item.find_element(By.CSS_SELECTOR,".amt > b")
                        unitQty_element = item.find_element(By.CSS_SELECTOR,".unitQty")
                        subinfo_element = item.find_element(By.CSS_SELECTOR,".subinfo > li")
                        infoDeli_element = item.find_element(By.CSS_SELECTOR,".infoDeli")
                        grade_element = item.find_element(By.CSS_SELECTOR,".grade")
                        shipping_amt_element = item.find_element(By.CSS_SELECTOR, "div.amtqty > div.infoDeli > b")
                        shipping_amt = convert_string_to_int_with_commas(shipping_amt_element.text)
                        
                        title_text = title_element.text
                        href_element_text = href_element.get_attribute("href")
                        amt_text = amt_element.text
                        unitQty_text = unitQty_element.text
                        subinfo_text = subinfo_element.text
                        infoDeli_text = infoDeli_element.text
                        grade_text = grade_element.text

                        product_amt = convert_string_to_int_with_commas(amt_text)
                        
                        # print(title_text, amt_text, unitQty_text, infoDeli_text, subinfo_text, grade_text)

                        # 상세 페이지
                        if (not href_element_text in data['supplier_url']) and ((unitQty_text == individually_purchas_name) or (subinfo_text in business_individually_purchas_name)):
                            # 새로운 탭 열기
                            driver.execute_script("window.open('" + href_element_text + "', 'new_window')")
                            
                            driver.switch_to.window(driver.window_handles[1])
    
                            while True:
                                try:
                                    isSelected = int(input("해당 상품을 선택하시겠습니까? 0: No, 1: Yes"))
                                    if isSelected == 0:
                                        break
                                    
                                    # 새로운 탭에서 필요한 작업 수행
                                    seller_info_element = driver.find_element(By.CSS_SELECTOR,"#lBtnShowSellerInfo > b") #제조사
                                    
                                    item_title_element = driver.find_element(By.CSS_SELECTOR,"#lInfoItemTitle") #상품 이름
            
                                    # lInfoAmt_element = driver.find_elements(By.CSS_SELECTOR,"#lInfoBody > div.lInfoBody.lInfoRow.lSelected > table > tbody > tr.lInfoAmt > td > div > div") #상품 가격
                                    # lInfoAmt_text = ''
                                    # if len(lInfoAmt_element) <= 0 : 
                                    #     lInfoAmt_element = driver.find_elements(By.CSS_SELECTOR,"#lAmtSectionTbl > tbody > tr:nth-child(2) > td.lSelected") #상품 가격
                                    #     lInfoAmt_text = lInfoAmt_element[0].text
                                    # else:
                                    #     lInfoAmt_text = lInfoAmt_element[0].text.split('원')[0]
                                    # lInfoAmt = convert_string_to_int_with_commas(lInfoAmt_text)
            
                                    
                                    min_qty_element = driver.find_element(By.CSS_SELECTOR,"#lInfoBody > div.lInfoBody.lInfoRow.lSelected > table > tbody > tr.lInfoPurchase > td > b") #최소 구매 수량
            
                                    lInfoQty_element = driver.find_element(By.CSS_SELECTOR,"#lInfoBody > div.lInfoBody.lInfoRow.lSelected > table > tbody > tr.lInfoQty > td") #제고 수량
            
                                    quest_element = driver.find_elements(By.CSS_SELECTOR,"#lInfoBody > div.lInfoBody.lInfoRow.lSelected > table > tbody > tr.lInfoPurchase > td > div > a") #훙정하기
                                    is_use_phto = driver.find_element(By.CSS_SELECTOR,"#lInfoView > div.lInfoViewSubWrap > form > table > tbody > tr > td.lInfoViewSubTd2 > div.lInfoViewImgUse > div:nth-child(1) > b") #사진 사용 여부,
    
                                    product_option_elements = driver.find_elements(By.CSS_SELECTOR,"#lPayInfoBody > table > tbody > tr > td > div.lOptRes > .pOptResultList > div") # 옵션
    
                                    if len(product_option_elements) <= 0:
                                        raise Exception("옵션를 선택해주세요.")
                               
                                    quest_text = ''
                                    if len(quest_element) > 0 :
                                        quest_text = quest_element[0].text
                                    else:
                                        quest_text = ''
            
                                    thum_img_element = driver.find_element(By.CSS_SELECTOR,"#lThumbImg") #썸네일 이미지
                                    detail_img_element = driver.find_element(By.CSS_SELECTOR," #lInfoViewItemContents img") #상세 이미지
            
                                    seller_info_text = seller_info_element.text
                                    item_title_text = item_title_element.text 
                                    min_qty_text = min_qty_element.text
                                    lInfoQty_text = lInfoQty_element.text
                                    thum_img_src = thum_img_element.get_attribute('src')
                                    detail_img_src = detail_img_element.get_attribute('src')
                                    is_use_phto_text = is_use_phto.text
                                    
    
                                    # 데이터 입력
                                    category_data = select_cloth_category(s_cloth_category)
    
                                    product_code_num = create_product_code(1,category_data,len(data['product_code']))
    
                                    option_list = []
                                    
                                    for index, product_option_element in enumerate(product_option_elements):
                                        option_value = " ".join(product_option_element.find_element(By.CSS_SELECTOR, ".pOptResultNames").text.split(' / '))
                                        convert_string_to_int_with_commas(product_option_element.find_element(By.CSS_SELECTOR, ".pOptResultTotal").text.split('원')[0])
                                        product_option_data_copy = product_option_data.copy()
                                        product_option_data_copy['product_id'] = product_code_num
                                        product_option_data_copy['sku_code'] = str(product_code_num) + str(index)
                                        product_option_data_copy['option_key'] = option_key['TYPE']
                                        product_option_data_copy['option_value'] = get_item_name(option_value)
                                        product_option_data_copy['option_sequence'] = index

                                        option_num_element = product_option_element.find_element(By.CSS_SELECTOR,"div.pOptResultInput > div.pOptResultQty > input[type=text]") # 옵션 개수

                                        option_num = option_num_element.get_attribute("value")
                                        print("옵션 개수:",option_num)
                                        
                                        product_option_data_copy['product_qty'] = option_num
                                        product_option_data_copy['max_order_qty'] = 10
                                        product_option_data_copy['min_order_qty'] = 1
                                        product_option_data_copy['option_extra_amount'] = 0
                                        product_option_data_copy['is_exposed'] = 1
                                        option_list.append(
                                            product_option_data_copy
                                        )
                                    
                                    brand_name_data = find_brand_by_name(brand_name)
    
                                    data_frame_copy = data_frame.copy()
                                    data_frame_copy['id'] = len(data['id']) + 1
                                    data_frame_copy['isUsePhoto'] = is_use_phto_text == use_photo_title
                                    data_frame_copy['product_category_id'] = category_data['id']
                                    data_frame_copy['product_brand_id'] = brand_name_data['id']
                                    data_frame_copy['productOptions'] = option_list
                                    data_frame_copy['name'] = get_item_name(item_title_text)
                                    data_frame_copy['product_code'] = product_code_num
                                    data_frame_copy['shipping_info_id'] = 1 
                                    data_frame_copy['brand_name'] = brand_name_data['ko_name']
                                    data_frame_copy['manufacturer_name'] = seller_info_text 
                                    data_frame_copy['supplier_name'] = '도매꾹'
                                    data_frame_copy['supplier_url'] = href_element_text
                                    data_frame_copy['supply_price'] = product_amt + shipping_amt

                                    reward_rate_ = check_range(float(input('소비자 리워드 비율을 넣어주세요.')),consumer_reward_rate_range)
                                    data_frame_copy['consumer_reward_rate'] = reward_rate_

                                    sale_price_ = get_sale_price(data_frame_copy['supply_price'],reward_rate_)
                                    print(product_amt)
                                    print("판매 가격:",sale_price_)
                
                                    
                                    discountrate_ = check_range(float(input('할인 비율을 넣어주세요.')),sale_rate_range)
                                    
                                    
                                    data_frame_copy['discount_rate'] = discountrate_
                                    data_frame_copy['sale_price'] = sale_price_
                                    data_frame_copy['consumer_price'] = round_ten(sale_price_ / (1 - discountrate_))
    
                                    data_frame_copy['tax_status'] = tax_status['TAX']
                                    data_frame_copy['business_reward_rate'] = 0
                                    data_frame_copy['image_path'] = thum_img_src
                                    data_frame_copy['detail_image_path'] = detail_img_src
                                    data_frame_copy['product_status'] = product_status['ACTIVE']
                                    data_frame_copy['product_priority'] = get_priority(data)
                                    data_frame_copy['is_exposed'] = 1
                                    data_frame_copy['is_available_multiple_option'] = 0
                                    data_frame_copy['sale_alternatives'] = '다 팔렸습니다.'
                                    data_frame_copy['prod_obs_ctrl_num'] = 0
                                    data_frame_copy['created_at'] = CREATED_AT
                                    data_frame_copy['updated_at'] = UPDATED_AT
                                    data_frame_copy['tax_rate'] = PG_TAX_PERCENT
                                    data_frame_copy['margin_rate'] = MARGIN_RATE
    
                                    for key, value in data_frame_copy.items():
                                        data[key].append(value)
                                    
                                    # print(seller_info_text,item_title_text,min_qty_text,lInfoQty_text,quest_text,thum_img_src ,detail_img_src, lInfoAmt_text, quest_text)
                                    print("\n")
                                    break
    
                                except Exception as e:
                                    print("오류가 발생했습니다.",e)
                                    is_obs = float(input("한번 더 보시겠습니까? (0: No, 1: Yes, -1: Exit)"))
                                    if is_obs == 0:
                                        break
                                    if is_obs == -1:
                                        # 새로운 탭 닫기
                                        driver.close()
                                        
                                        # 이전 탭으로 전환
                                        driver.switch_to.window(driver.window_handles[0])
                                        return
                                
                            # 새로운 탭 닫기
                            driver.close()
                            
                            # 이전 탭으로 전환
                            driver.switch_to.window(driver.window_handles[0])
                    except Exception as e:
                        pass

In [36]:
main_func()

https://mirror1.domeggook.com/main/item/itemList.php?mnp=&mxp=&mnq=&mxq=&dfos=false&sgd=1&sw=&so=da&ca=02_01_03_02_00&pg=1


해당 상품을 선택하시겠습니까? 0: No, 1: Yes 0
해당 상품을 선택하시겠습니까? 0: No, 1: Yes 1


오류가 발생했습니다. 옵션를 선택해주세요.


한번 더 보시겠습니까? (0: No, 1: Yes, -1: Exit) 1
해당 상품을 선택하시겠습니까? 0: No, 1: Yes 1


id: 14 종류: SWEATSHIRT


...

상품 이름: 여성 여자 맨투맨 라운드 티셔츠 긴팔 데일리 캐주얼 빅사이즈 큰사이즈 간절기 일상 무지 심플 티


해당 상품 이름으로 하시곘습니까? 1: Yes, 0: No 맨투맨 라운드 티셔츠
해당 상품 이름으로 하시곘습니까? 1: Yes, 0: No 1
소비자 리워드 비율을 넣어주세요. 0.03


10000
판매 가격: 15030


할인 비율을 넣어주세요. 0.5
우선 순위 번호를 입력해주세요. (-1: 무작위 수, 그외 숫자: 직접입력 -1


KeyboardInterrupt: Interrupted by user

In [ ]:
pandas_data = pd.DataFrame(data)

In [ ]:
pandas_data

In [ ]:
pandas_data.to_csv(path_or_buf=cloth_data_path)

In [37]:
data

{'id': [1, 2],
 'isUsePhoto': [True, True],
 'product_category_id': [14, 14],
 'product_brand_id': [16, 14],
 'productOptions': [[{'product_id': 101400000001,
    'sku_code': '1014000000010',
    'option_key': 'TYPE',
    'option_value': 'SIS-110 카라루즈맨투맨 화이트 FREE(2xl)',
    'option_sequence': 0,
    'product_qty': '49999',
    'max_order_qty': 10,
    'min_order_qty': 1,
    'option_extra_amount': 0,
    'is_exposed': 1},
   {'product_id': 101400000001,
    'sku_code': '1014000000011',
    'option_key': 'TYPE',
    'option_value': 'SIS-110 카라루즈맨투맨 블랙 FREE(2xl)',
    'option_sequence': 1,
    'product_qty': '49999',
    'max_order_qty': 10,
    'min_order_qty': 1,
    'option_extra_amount': 0,
    'is_exposed': 1}],
  [{'product_id': 101400000002,
    'sku_code': '1014000000020',
    'option_key': 'TYPE',
    'option_value': '블랙',
    'option_sequence': 0,
    'product_qty': '49999',
    'max_order_qty': 10,
    'min_order_qty': 1,
    'option_extra_amount': 0,
    'is_exposed': 1},
   

In [54]:
def list_to_string(data):
    result = ""
    for item in data:
        if isinstance(item, str):
            result += f"'{item}', "
        else:
            result += f"{item}, "
    return result[:-2]  # 마지막 쉼표와 공백 제거

### SQL 파일 작성

In [ ]:
# f"INSERT INTO `{db_name}`.`{table_name}` ({key_data_name}) VALUES ({value_data_name});"

## ProductBrand SQL 파일 작성

In [2]:
db_name = "bucl_db"
table_name = "product_brand"

sql_template = {
    "created_at": "",
    "updated_at": "",
    "product_brand_id": "",
    "brand_co_name": "",
    "brand_profile_path": "",
    "brand_name": "",
    "brand_desc": "",
    "brand_img_list_path":""
}

In [ ]:
INSERT INTO `bucl_db`.`product_brand` (`product_brand_id`, `created_at`, `updated_at`, `brand_co_name`, `brand_profile_path`, `brand_name`, `brand_desc`, `brand_img_list_path`) VALUES ('16', '2024-03-11 13:59:00.706416', '2024-03-11 13:59:00.706416', '벨라', 'https://s3.ap-northeast-2.amazonaws.com/image.pixpergram.com/brand/profile/sjkgjrijtgiowejklfajd.png', '벨라', '이쁜 당신을 위한 5가지 스타일', 'https://s3.ap-northeast-2.amazonaws.com/image.pixpergram.com/brand/asset/jdgkjkldfjkaldjfkladla.png');


In [4]:
brand_name

[{'name': 'Styloo', 'ko_name': '스타이루', 'id': 1},
 {'name': 'Serenity', 'ko_name': '세레니티', 'id': 2},
 {'name': 'Canvas Style', 'ko_name': '캔버스 스타일', 'id': 3},
 {'name': 'Lumina', 'ko_name': '루미나', 'id': 4},
 {'name': 'Peerless', 'ko_name': '피어리스', 'id': 5},
 {'name': 'Aether', 'ko_name': '에테르', 'id': 6},
 {'name': 'Bloomer', 'ko_name': '블루머', 'id': 7},
 {'name': 'Iris Eye', 'ko_name': '아이리스 아이', 'id': 8},
 {'name': 'Sweet Ander', 'ko_name': '스위트 앤더', 'id': 9},
 {'name': 'Prismer', 'ko_name': '프리즈머', 'id': 10},
 {'name': 'Sage Style', 'ko_name': '세이지 스타일', 'id': 11},
 {'name': 'Violet Bloom', 'ko_name': '바이올렛 블룸', 'id': 12},
 {'name': 'Clover Kiss', 'ko_name': '클로버 키스', 'id': 13},
 {'name': 'Cream Candy', 'ko_name': '크림 캔디', 'id': 14},
 {'name': 'Pluto', 'ko_name': '플루토', 'id': 15},
 {'name': 'Vela', 'ko_name': '벨라', 'id': 16},
 {'name': 'Twila', 'ko_name': '트와일라', 'id': 17},
 {'name': 'Velvetine', 'ko_name': '벨베틴', 'id': 18},
 {'name': 'AvaLys', 'ko_name': '아바리스', 'id': 19},
 {'name': '

In [ ]:
for pd_d in pandas_data.to_dict(orient='records'):
    for key, value in sql_template.items():
        try:
            sql_template[key] = pd_d[key]
        except Exception as e:
            print(key)
            pass
    key_data_name = ', '.join([f"`{item}`" for item in list(sql_template.keys())])
    value_data_name = list_to_string(list(sql_template.values()))
    sql_ = f"INSERT INTO `{db_name}`.`{table_name}` ({key_data_name}) VALUES ({value_data_name});"
    sql_list.append(sql_)

## Product SQL 파일 작성

In [57]:
db_name = "bucl_db"
table_name = "product"

sql_template = {
    "created_at": "",
    "updated_at": "",
    "brand_name": "",
    "business_reward_rate": "",
    "consumer_price": "",
    "consumer_reward_rate": "",
    "detail_image_path": "",
    "discount_rate": "",
    "image_path": "",
    "is_available_multiple_option": "",
    "is_exposed": "",
    "manufacturer_name": "",
    "margin_rate": "",
    "name": "",
    "product_code": "",
    "product_priority": "",
    "product_status": "",
    "sale_alternatives": "",
    "sale_price": "",
    "supplier_name": "",
    "supply_price": "",
    "tax_rate": "",
    "tax_status": "",
    "product_category_id": "",
    "shipping_info_id": "",
    "prod_obs_ctrl_num": "",
    "product_brand_id": "",
    "supplier_url": ""
}

In [43]:
sql_template

{'created_at': '2024-03-29 00:00:00.000000',
 'updated_at': '2024-03-29 00:00:00.000000',
 'brand_name': '크림 캔디',
 'business_reward_rate': 0,
 'consumer_price': 23130.0,
 'consumer_reward_rate': 0.03,
 'detail_image_path': 'https://shop-phinf.pstatic.net/20220504_10/1651636690499cgHKS_JPEG/1-vert.jpg?type=w860',
 'discount_rate': 0.5,
 'image_path': 'http://cdn1.domeggook.com/upload/item/2023/08/25/16929030530D40404F71E80D988AFFAA/16929030530D40404F71E80D988AFFAA_img_760?hash=f361ded5d8b9f8d5f0aaf9dfac685708',
 'is_available_multiple_option': 0,
 'is_exposed': 1,
 'manufacturer_name': '루퍼시',
 'margin_rate': 0.1,
 'name': '여성 여자 맨투맨 라운드 티셔츠 긴팔 데일리 캐주얼 빅사이즈 큰사이즈 간절기 일상 무지 심플 티',
 'product_code': 101400000002,
 'product_priority': 61,
 'product_status': 'ACTIVE',
 'sale_alternatives': '다 팔렸습니다.',
 'sale_price': 11560,
 'supplier_name': '도매꾹',
 'supply_price': 13000,
 'tax_rate': 0.018,
 'tax_status': 'TAX',
 'product_category_id': 14,
 'shipping_info_id': 1,
 'prod_obs_ctrl_num': 0,
 'pro

In [63]:
sql_list = []

In [64]:
for pd_d in pandas_data.to_dict(orient='records'):
    for key, value in sql_template.items():
        try:
            sql_template[key] = pd_d[key]
        except Exception as e:
            print(key)
            pass
    key_data_name = ', '.join([f"`{item}`" for item in list(sql_template.keys())])
    value_data_name = list_to_string(list(sql_template.values()))
    sql_ = f"INSERT INTO `{db_name}`.`{table_name}` ({key_data_name}) VALUES ({value_data_name});"
    sql_list.append(sql_)


In [71]:
with open("./data/sql/data.sql","w") as f:
    f.write("\n".join(sql_list))